In [2]:
%load_ext autoreload
%autoreload 2
main_dir = r'f:\Projects\Prvate\Baydin'

data_path = r'E:/Kaggle/06-LabelEncodingSuperLast'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *

import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Code to make pipeline node

In [3]:
data = pd.read_pickle(f'{data_path}/data.pkl')

In [4]:
X = data[data.isFraud>=0].drop('isFraud',axis=1)
test = data[data.isFraud<0].drop('isFraud',axis=1)
y= data[data.isFraud>=0].isFraud

In [13]:
import yaml
categorical_features = yaml.load(open(f'{data_path}/categorical_columns.yaml','r'))
numerical_features = yaml.load(open(f'{data_path}/numerical_columns.yaml','r'))
bad_features = yaml.load(open(f'{data_path}/bad_for_training_features.yaml','r'))

categorical_features = [f for f in categorical_features if f not in bad_features]
all_features =  [f for f in X.columns if f not in bad_features]

In [28]:
from BayDS.lib.training import *

In [7]:
drop_big_categorical_features = []
for col in categorical_features:
    if len(data[col].value_counts()) > 50:
        drop_big_categorical_features.append(col)
        print(col, len(data[col].value_counts()))

card_id 19226
new_card_id 144827
OSVersion 88
device_version 380
BrowserVersion 54
id_13 56
id_17 128
id_19 569
id_20 548
id_30 88
id_31 173
id_33 462
DeviceInfo 2800
P_emaildomain 61
R_emaildomain 61
card1 9118
card2 497
card3 97
card5 84
addr1 442
device_version 380
id_02__id_20 270750
id_02__D8 258079
D11__DeviceInfo 3680
DeviceInfo__P_emaildomain 8978
P_emaildomain__C2 6793
card2__dist1 31408
card1__card5 18808
card2__id_20 9240
card5__P_emaildomain 1574
addr1__card1 55370
uid 11014
uid2 11442
uid3 52025
uid4 129426
uid5 91344
bank_type 726
product_type 30362


In [8]:
# X.drop(features_to_drop,axis=1,inplace=True)
# test.drop(features_to_drop,axis=1,inplace=True)
# use_for_train_features =  [f for f in all_features if f not in drop_big_categorical_features]

In [9]:
# categorical_features =  [f for f in categorical_features if f not in drop_big_categorical_features]

In [14]:
for f in categorical_features:
    if f not in X.columns:
        print(f"Lost {f}")
#         categorical_features.remove(f)
categorical_features = [f for f in categorical_features if f in X.columns]

In [15]:
for f in numerical_features:
    if f not in X.columns:
        print(f"Lost {f}")
#         numerical_features.remove(f)
numerical_features = [f for f in numerical_features if f in X.columns]

In [16]:
for col in X.columns:
    if str(X[col].dtype) == 'datetime64[ns]':
        print (col, X[col].dtype)
        X[col] = pd.to_timedelta(X[col]).map(lambda qq:qq.days)
        test[col] = pd.to_timedelta(test[col]).map(lambda qq:qq.days)
#         X[col] = X[col].astype(datetime.datetime)

In [17]:
from sklearn.model_selection import GroupKFold

NFOLDS = 6
folds = GroupKFold(n_splits=NFOLDS)


In [26]:
params = {
            'n_job': -1,
            'max_depth': 8,
            'learning_rate': 0.01,
            'colsample_bytree': 0.5,
            'random_state': 12,
            'tree_method': 'gpu_hist',
            'objective': 'binary:logistic',
            "eval_metric":'auc'
         }
train_options = {
        "model_type":'xgb',
        "params": params,
        'early_stopping_rounds': 500,
        'n_estimators': 10000,
        'averaging': 'usual',
        'use_groups': False,
        'fold_name': folds.__class__.__name__,
        'n_splits': NFOLDS
    }

In [30]:
result_dict = train_model_classification_vb( 
#                                              splits=1,
                                             X=X, X_test=test, columns=use_for_train_features,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             categorial_columns = categorical_features,
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
#                                              train_0_sample_coef=0.2, train_1_sample_coef=2.0,
                                             n_jobs=-1, groups=X['DT_M'])


Fold 1 started at Tue Oct  1 10:34:45 2019
[0]	train-auc:0.838432	valid_data-auc:0.754978
Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train until valid_data-auc hasn't improved in 500 rounds.
[500]	train-auc:0.95798	valid_data-auc:0.884813
[1000]	train-auc:0.978554	valid_data-auc:0.904021
[1500]	train-auc:0.986519	valid_data-auc:0.909922
[2000]	train-auc:0.991291	valid_data-auc:0.912341
[2500]	train-auc:0.994277	valid_data-auc:0.913392
[3000]	train-auc:0.996292	valid_data-auc:0.913617
[3500]	train-auc:0.997758	valid_data-auc:0.913689
Stopping. Best iteration:
[3369]	train-auc:0.997419	valid_data-auc:0.913753

Fold 2 started at Tue Oct  1 10:44:49 2019
[0]	train-auc:0.817347	valid_data-auc:0.810479
Multiple eval metrics have been passed: 'valid_data-auc' will be used for early stopping.

Will train until valid_data-auc hasn't improved in 500 rounds.
[500]	train-auc:0.956033	valid_data-auc:0.915546
[1000]	train-auc:0.977667	valid_data-a

In [26]:
import pickle
pickle.dump(result_dict,open(f'{data_path}/results_dict_xgb-work-1.pkl','wb'))

In [28]:
pd.DataFrame(result_dict['oof'], columns=['isFraud'], index=X.index).to_csv(f'{data_path}/oof_lightgbm-work-1.csv')

pd.DataFrame(result_dict['prediction'], columns=['isFraud'], index=test.index).to_csv(f'{data_path}/prediction_lightgbm-work-1.csv')

sample_submission = pd.read_csv(data_path + '/sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_dict['prediction'], columns=['isFraud'], index=test.index)

sample_submission['isFraud'] = sub1#*0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv(f'{data_path}/lightgbm-work-1_sub.csv')

FileNotFoundError: [Errno 2] File b'E:/Kaggle/05-LabelEncoded-last/sample_submission.csv' does not exist: b'E:/Kaggle/05-LabelEncoded-last/sample_submission.csv'